# json 파일 생성 / train tesst split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sentencepiece as spm
import pandas as pd
import json

In [ ]:
# 파일 읽기
df = pd.read_csv("/content/drive/MyDrive/Boaz/미니플젝1/data/news_edited.csv", sep=",", engine="python")

# DataFrame의 모든 열 이름 출력
print(df.columns)

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split 함수를 사용해 데이터 분할
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)  # test_size는 20%, random_state는 재현 가능성을 위해 설정d

train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [ ]:
vocab_file = "news2.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [ ]:
""" train data 준비 """
def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep=",", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows():
            text = row["text"]
            if type(text) != str:
                continue
            instance = { "id": row["Unnamed: 0"], "doc": vocab.encode_as_pieces(text), "label": row["label"] }
            f.write(json.dumps(instance))
            f.write("\n")

In [ ]:
prepare_train(vocab, "train_data.csv", "news_train.json")
prepare_train(vocab, "test_data.csv", "news_test.json")

# model에 없는 단어 제외

In [2]:
!pip install sentencepiece

In [5]:
import sentencepiece as spm

# 모델의 SentencePiece 파일 경로
model_path = 'news2.model'

# SentencePiece 모델 로드
sp = spm.SentencePieceProcessor(model_file=model_path)

# 어휘 목록 추출 (index와 함께)
vocab_words = [sp.id_to_piece(i) for i in range(sp.get_piece_size())]

# 어휘 목록을 집합으로 변환
vocab_words_set = set(vocab_words)

In [7]:
import json

# JSON 파일 경로
json_path = 'news_test.json'

# JSON 파일 읽기
with open(json_path, 'r') as file:
    data = [json.loads(line) for line in file]

# 모델 어휘에 포함된 단어만 필터링
filtered_data = []
for entry in data:
    if isinstance(entry["doc"], list):
        # 리스트를 문자열로 변환 후 단어 추출
        text = ' '.join(entry["doc"])
    else:
        text = entry["doc"]

    # 텍스트에서 단어 추출 (단어는 공백으로 분리된다고 가정)
    words = text.split()

    # 모델 어휘에 포함된 단어만 필터링
    filtered_words = [word for word in words if word in vocab_words]

    if filtered_words:
        filtered_data.append({
            "id": entry["id"],
            "doc": filtered_words,  # 필터링된 단어로 업데이트
            "label": entry["label"]
        })


In [8]:
import json

# 필터링된 데이터를 새로운 JSON 파일로 저장
filtered_json_path = 'news2_test.json'

with open(filtered_json_path, 'w') as f:
    for entry in filtered_data:
        f.write(json.dumps(entry) + "\n")


In [9]:
import json

# JSON 파일 경로
json_path = 'news_train.json'

# JSON 파일 읽기
with open(json_path, 'r') as file:
    data = [json.loads(line) for line in file]

# 모델 어휘에 포함된 단어만 필터링
filtered_data = []
for entry in data:
    if isinstance(entry["doc"], list):
        # 리스트를 문자열로 변환 후 단어 추출
        text = ' '.join(entry["doc"])
    else:
        text = entry["doc"]

    # 텍스트에서 단어 추출 (단어는 공백으로 분리된다고 가정)
    words = text.split()

    # 모델 어휘에 포함된 단어만 필터링
    filtered_words = [word for word in words if word in vocab_words]

    if filtered_words:
        filtered_data.append({
            "id": entry["id"],
            "doc": filtered_words,  # 필터링된 단어로 업데이트
            "label": entry["label"]
        })


In [10]:
import json

# 필터링된 데이터를 새로운 JSON 파일로 저장
filtered_json_path = 'news2_train.json'

with open(filtered_json_path, 'w') as f:
    for entry in filtered_data:
        f.write(json.dumps(entry) + "\n")
